In [1]:
setwd(fs::path_abs("~/Local_Workspace/TesiMag"))
library(ggplot2, warn.conflicts = FALSE)
library(readr, warn.conflicts = FALSE)
library(arrow, warn.conflicts = FALSE)
library(dplyr, warn.conflicts = FALSE)
library(tidyr, warn.conflicts = FALSE)

options(repr.plot.width = 9, repr.plot.res = 300)


source("src/paths/paths.R")
path.piemonte <- file.path(path.ds, "ARPA", "PIEMONTE")
path.dwnl <- file.path(path.piemonte, "site-data")


In [2]:
open_dataset(path.dwnl, format = "parquet") |>
    select(date = data, T_MAX = tmax, T_MIN = tmin, tclasse, identifier = id_punto) |>
    mutate(tclasse = cast(tclasse, utf8()), identifier = cast(identifier, utf8())) |>
    map_batches(
        ~ pivot_longer(. |> collect(), cols = c(T_MAX, T_MIN), names_to = "variable", values_to = "value") |>
            as_record_batch()
    ) |>
    arrange(variable, identifier, date) |>
    # write_parquet("test.parquet")
    write_dataset(file.path(path.piemonte, "dataset"), format = "feather", partitioning = "variable")


In [ ]:
open_dataset(file.path(path.piemonte, "dataset"), format = "feather") |>
    filter(variable == "T_MIN") |>
    collect()


In [11]:
library(sf, warn.conflicts = FALSE)
read_csv_arrow(file.path(path.piemonte, "metadata.csv"), col_select = c(data_inizio_sensore, data_fine_sensore, quota_da_pc, quota_stazione, esposizione, tipo_staz, sigla_prov, comune, denominazione, indirizzo_localita, identifier = id_punto_misura, lon = longitudine_e_wgs84_d, lat = latitudine_n_wgs84_d)) |>
    st_as_sf(coords = c("longitudine_e_wgs84_d", "latitudine_n_wgs84_d"), crs = "WGS84") |>
    write_sf(file.path(path.piemonte, "metadata.geojson"))
